<br>
### Load packages

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from datetime import datetime
from scipy.stats import skew  # for some statistics
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error
from mlxtend.regressor import StackingCVRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
import scipy.stats as stats
import sklearn.linear_model as linear_model
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import os
print(os.listdir("../input"))

import warnings
warnings.filterwarnings('ignore')

# Any results you write to the current directory are saved as output.

### Load data

In [ ]:
train = pd.read_csv('/kaggle/input/ml-for-oceanography/train.csv')
test = pd.read_csv('/kaggle/input/ml-for-oceanography/test1.csv')
#Semi supervised learning
# top_sub = pd.read_csv('/kaggle/input/oceam-ml-top-submission-66/submission18(added kernel regression).csv')
# semi_train = test
# semi_train['WVHT(m)']= top_sub['WVHT(m)']
# train = train.append(semi_train)
print ("Data is loaded!")

In [ ]:
print ("Train: ",train.shape[0],"sales, and ",train.shape[1],"features")
print ("Test: ",test.shape[0],"sales, and ",test.shape[1],"features")

In [ ]:
train.head()

In [ ]:
test.head()

# EDA

There are 1460 instances of training data and 1460 of test data. Total number of attributes equals 81, of which 36 is quantitative, 43 categorical + ID and WVHT(m).

**Quantitative:** 1stFlrSF, 2ndFlrSF, 3SsnPorch, BedroomAbvGr, BsmtFinSF1, BsmtFinSF2, BsmtFullBath, BsmtHalfBath, BsmtUnfSF, EnclosedPorch, Fireplaces, FullBath, GarageArea, GarageCars, GarageYrBlt, GrLivArea, HalfBath, KitchenAbvGr, LotArea, LotFrontage, LowQualFinSF, MSSubClass, MasVnrArea, MiscVal, MoSold, OpenPorchSF, OverallCond, OverallQual, PoolArea, ScreenPorch, TotRmsAbvGrd, TotalBsmtSF, WoodDeckSF, YearBuilt, YearRemodAdd, YrSold

**Qualitative:** Alley, BldgType, BsmtCond, BsmtExposure, BsmtFinType1, BsmtFinType2, BsmtQual, CentralAir, Condition1, Condition2, Electrical, ExterCond, ExterQual, Exterior1st, Exterior2nd, Fence, FireplaceQu, Foundation, Functional, GarageCond, GarageFinish, GarageQual, GarageType, Heating, HeatingQC, HouseStyle, KitchenQual, LandContour, LandSlope, LotConfig, LotShape, MSZoning, MasVnrType, MiscFeature, Neighborhood, PavedDrive, PoolQC, RoofMatl, RoofStyle, SaleCondition, SaleType, Street, Utilities,

In [ ]:
quantitative = [f for f in train.columns if train.dtypes[f] != 'object']
quantitative.remove('WVHT(m)')
quantitative.remove('ID')
qualitative = [f for f in train.columns if train.dtypes[f] == 'object']

In [ ]:
quantitative

In [ ]:
qualitative

19 attributes have missing values, 5 over 50% of all data. Most of times NA means lack of subject described by attribute, like missing pool, fence, no garage and basement.

In [ ]:
y = train['WVHT(m)']
plt.figure(1); plt.title('Johnson SU')
sns.distplot(y, kde=False, fit=stats.johnsonsu)
plt.figure(2); plt.title('Normal')
sns.distplot(y, kde=False, fit=stats.norm)
plt.figure(3); plt.title('Log Normal')
sns.distplot(y, kde=False, fit=stats.lognorm)

It is apparent that WVHT(m) doesn't follow normal distribution, so before performing regression it has to be transformed. While log transformation does pretty good job, best fit is unbounded Johnson distribution.

In [ ]:
test_normality = lambda x: stats.shapiro(x.fillna(0))[1] < 0.01
normal = pd.DataFrame(train[quantitative])
normal = normal.apply(test_normality)
print(not normal.any())

Also none of quantitative variables has normal distribution so these should be transformed as well.

**Spearman correlation** is better to work with in this case because it picks up relationships between variables even when they are nonlinear. OverallQual is main criterion in establishing house price. Neighborhood has big influence, partially it has some intrisinc value in itself, but also houses in certain regions tend to share same characteristics (confunding) what causes similar valuations.

In [ ]:
def encode(frame, feature):
    ordering = pd.DataFrame()
    ordering['val'] = frame[feature].unique()
    ordering.index = ordering.val
    ordering['spmean'] = frame[[feature, 'WVHT(m)']].groupby(feature).mean()['WVHT(m)']
    ordering = ordering.sort_values('spmean')
    ordering['ordering'] = range(1, ordering.shape[0]+1)
    ordering = ordering['ordering'].to_dict()
    
    for cat, o in ordering.items():
        frame.loc[frame[feature] == cat, feature+'_E'] = o
    
qual_encoded = []
for q in qualitative:  
    encode(train, q)
    qual_encoded.append(q+'_E')
print(qual_encoded)

In [ ]:
def spearman(frame, features):
    spr = pd.DataFrame()
    spr['feature'] = features
    spr['spearman'] = [frame[f].corr(frame['WVHT(m)'], 'spearman') for f in features]
    spr = spr.sort_values('spearman')
    plt.figure(figsize=(6, 0.25*len(features)))
    sns.barplot(data=spr, y='feature', x='spearman', orient='h')
    
features = quantitative + qual_encoded
#spearman(train, features)

In [ ]:
plt.figure(1)
corr = train[quantitative+['WVHT(m)']].corr()
sns.heatmap(corr)
plt.figure(2)
corr = train[qual_encoded+['WVHT(m)']].corr()
sns.heatmap(corr)
plt.figure(3)
corr = pd.DataFrame(np.zeros([len(quantitative)+1, len(qual_encoded)+1]), index=quantitative+['WVHT(m)'], columns=qual_encoded+['WVHT(m)'])
for q1 in quantitative+['WVHT(m)']:
    for q2 in qual_encoded+['WVHT(m)']:
        corr.loc[q1, q2] = train[q1].corr(train[q2])
sns.heatmap(corr)

### Simple clustering

In [ ]:
features = quantitative + qual_encoded

# Models

### Data processing

In [ ]:
train.drop(['ID'], axis=1, inplace=True)
test.drop(['ID'], axis=1, inplace=True)

In [ ]:
#train = train[train.GrLivArea < 4500]
train.reset_index(drop=True, inplace=True)
#train["WVHT(m)"] = np.log1p(train["WVHT(m)"])
y = train['WVHT(m)'].reset_index(drop=True)
y = np.log1p(y)

### Features

In [ ]:
train_features = train.drop(['WVHT(m)'], axis=1)
test_features = test
features = pd.concat([train_features, test_features]).reset_index(drop=True)

In [ ]:
features.shape

In [ ]:
features.head()

In [ ]:
numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerics2 = []
for i in features.columns:
    if features[i].dtype in numeric_dtypes:
        numerics2.append(i)
skew_features = features[numerics2].apply(lambda x: skew(x)).sort_values(ascending=False)

high_skew = skew_features[skew_features > 0.5]
skew_index = high_skew.index

for i in skew_index:
    features[i] = boxcox1p(features[i], boxcox_normmax(features[i] + 1))

In [ ]:
features.head()

In [ ]:
import numpy as np



import pandas as pd
import numpy as np
# Assuming features is a pandas DataFrame with the features
window_size = 12  # in hours
# Define the columns to create rolling features for
rolling_cols = ['WDIR(degT)', 'WSPD(m/s)', 'GST(m/s)', 'DPD(sec)', 'APD(sec)', 'PRES(hPa)', 'ATMP(degC)', 'WTMP(degC)', 'DEWP(degC)']
# Create rolling averages for each column
for col in rolling_cols:
    features[f'{col}_rolling_mean_{window_size}h'] = features[col].rolling(window_size, min_periods=1).mean()
    
# # Create rolling standard deviations for each column
# for col in rolling_cols:
#     features[f'{col}_rolling_std_{window_size}h'] = features[col].rolling(window_size, min_periods=1).std()
    
# # Create rolling minimums for each column
# for col in rolling_cols:
#     features[f'{col}_rolling_min_{window_size}h'] = features[col].rolling(window_size, min_periods=1).min()
    
# # Create rolling maximums for each column
# for col in rolling_cols:
#     features[f'{col}_rolling_max_{window_size}h'] = features[col].rolling(window_size, min_periods=1).max()
# Create rolling percentiles for each column
for col in rolling_cols:
    for pct in [0.25, 0.5, 0.75]:
        features[f'{col}_rolling_pct_{pct}_{window_size}h'] = features[col].rolling(window_size, min_periods=1).quantile(pct)

# Add hour-of-day features
features['hour_sin'] = np.sin(2*np.pi*features['hh']/24)
features['hour_cos'] = np.cos(2*np.pi*features['hh']/24)
#features = features.drop('hh', axis=1)

# Add wind direction feature
features['WDIR_sin'] = np.sin(2*np.pi*features['WDIR(degT)']/360)
features['WDIR_cos'] = np.cos(2*np.pi*features['WDIR(degT)']/360)
#features = features.drop('WDIR(degT)', axis=1)

# Add wave direction feature
features['MWD_sin'] = np.sin(2*np.pi*features['MWD(degT)']/360)
features['MWD_cos'] = np.cos(2*np.pi*features['MWD(degT)']/360)
#features = features.drop('MWD(degT)', axis=1)

# Add temperature difference feature
features['temp_diff'] = features['ATMP(degC)'] - features['WTMP(degC)']
#features = features.drop(['ATMP(degC)', 'WTMP(degC)'], axis=1)

# Add pressure difference feature
features['pres_diff'] = features['PRES(hPa)'].diff()
features['pres_diff'].iloc[0] = 0
#features = features.drop('PRES(hPa)', axis=1)

# Add rolling mean feature
features['WSPD(m/s)_rolling_mean'] = features['WSPD(m/s)'].rolling(window=3, min_periods=1).mean()

# Add interaction features
features['WSPD_WDIR_interaction'] = features['WSPD(m/s)'] * features['WDIR_sin']
features['DPD_MWD_interaction'] = features['DPD(sec)'] * features['MWD_sin']
features['temp_pres_interaction'] = features['temp_diff'] * features['pres_diff']
features['WSPD_pres_interaction'] = features['WSPD(m/s)'] * features['pres_diff']
features['APD_WDIR_interaction'] = features['APD(sec)'] * features['WDIR_sin']
features['WSPD_DPD_interaction'] = features['WSPD(m/s)'] * features['DPD(sec)']


# Define more interaction features
features['WSPD_WDIR_interaction'] = features['WSPD(m/s)'] * np.sin(features['WDIR(degT)'])
features['DPD_MWD_interaction'] = features['DPD(sec)'] * np.sin(features['MWD(degT)'])
features['temp_pres_interaction'] = features['ATMP(degC)'] * features['PRES(hPa)']
features['WSPD_pres_interaction'] = features['WSPD(m/s)'] * features['PRES(hPa)']
features['APD_WDIR_interaction'] = features['APD(sec)'] * np.sin(features['WDIR(degT)'])
features['WSPD_DPD_interaction'] = features['WSPD(m/s)'] * features['DPD(sec)']
features['WDIR_PRES_interaction'] = np.sin(features['WDIR(degT)']) * features['PRES(hPa)']
features['WSPD_DirDev_interaction'] = features['WSPD(m/s)'] * np.sin(np.abs(features['WDIR(degT)'] - features['MWD(degT)']))
features['ATMP_WDIR_interaction'] = features['ATMP(degC)'] * np.sin(features['WDIR(degT)'])
features['WSPD_MWD_interaction'] = features['WSPD(m/s)'] * np.sin(features['MWD(degT)'])
features['DPD_WDIR_interaction'] = features['DPD(sec)'] * np.sin(features['WDIR(degT)'])
features['APD_PRES_interaction'] = features['APD(sec)'] * features['PRES(hPa)']
features['ATMP_DPD_interaction'] = features['ATMP(degC)'] * features['DPD(sec)']
features['WDIR_DirDev_interaction'] = np.sin(features['WDIR(degT)']) * np.sin(np.abs(features['WDIR(degT)'] - features['MWD(degT)']))
features['WSPD_ATMP_interaction'] = features['WSPD(m/s)'] * features['ATMP(degC)']
features['MWD_PRES_interaction'] = np.sin(features['MWD(degT)']) * features['PRES(hPa)']
features['DPD_PRES2_interaction'] = features['DPD(sec)'] * features['PRES(hPa)']**2
features['WSPD_WDIR2_interaction'] = features['WSPD(m/s)'] * np.sin(features['WDIR(degT)'])**2
features['APD_ATMP_interaction'] = features['APD(sec)'] * features['ATMP(degC)']
features['WDIR_temp_interaction'] = np.sin(features['WDIR(degT)']) * features['ATMP(degC)']
features['WSPD_PRES2_interaction'] = features['WSPD(m/s)'] * features['PRES(hPa)']**2
features['MWD_temp_interaction'] = np.sin(features['MWD(degT)']) * features['ATMP(degC)']
features['WSPD_DirDev2_interaction'] = features['WSPD(m/s)'] * np.sin(np.abs(features['WDIR(degT)'] - features['MWD(degT)']))**2
features['DPD_ATMP_interaction'] = features['DPD(sec)'] * features['ATMP(degC)']

                                             
# Compute wave height from period
features['wave_height_from_period'] = (features['DPD(sec)'] * 9.81) / (2 * np.pi)
# Compute wave speed and wavelength assuming water depth of 50m
features['wave_speed'] = 1.56 * (features['DPD(sec)']**0.5)
features['wavelength'] = features['wave_speed'] * features['DPD(sec)']

# Compute wave steepness
g = 9.81
features['wavelength1'] = features['APD(sec)']**2 * g / (2 * np.pi)
features['wave_steepness'] = features['wave_height_from_period'] / features['wavelength1']

# Compute u and v components of wind speed
features['u_wind'] = -features['WSPD(m/s)'] * np.sin(features['WDIR(degT)'] * np.pi / 180)
features['v_wind'] = -features['WSPD(m/s)'] * np.cos(features['WDIR(degT)'] * np.pi / 180)
# Compute wave power
rho = 1025
g = 9.81
features['wave_power_dpd'] = 0.5 * rho * g * features['wave_height_from_period']**2 * features['DPD(sec)']
features['wave_power_apd'] = 0.5 * rho * g * features['wave_height_from_period']**2 * features['APD(sec)']

# compute wave age
features['wave_age'] = features['APD(sec)']/((features['WSPD(m/s)']+1e-4)/0.71)

#Compute wave energy
features['wave_energy'] = 0.5*rho*g*features['wave_height_from_period']**2



# Add day-of-week features
features.update(features.fillna(0))

In [ ]:
features.shape

In [ ]:
final_features = pd.get_dummies(features).reset_index(drop=True)
final_features.shape

In [ ]:
final_features.head()

In [ ]:
X = final_features.iloc[:len(y), :]
X_sub = final_features.iloc[len(y):, :]
X.shape, y.shape, X_sub.shape

In [ ]:
X.shape, y.shape, X_sub.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Initialize the Random Forest Regressor model
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model on the training data
rf_regressor.fit(X, y)

# Print the feature importance
importance = rf_regressor.feature_importances_

import matplotlib.pyplot as plt

# Get the feature names
feature_names = list(features.columns)

# Plot the feature importances
plt.bar(feature_names, importance)
plt.title("Feature Importances")
plt.xlabel("Features")
plt.ylabel("Importance")
plt.show()

import pandas as pd

# Create a DataFrame with feature names and importances
feature_importances = pd.DataFrame({'feature': feature_names, 'importance': importance})

# Sort the features by importance in descending order
feature_importances = feature_importances.sort_values('importance', ascending = False).reset_index(drop=True)

# Print the sorted table
print(feature_importances)



In [ ]:
imp_feats = list(feature_importances['feature'][:20])

In [ ]:
imp_final_features = final_features[imp_feats]

In [ ]:
kfolds = KFold(n_splits=10, shuffle=True, random_state=42)

def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

def cv_rmse(model, X=X):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=kfolds))
    return (rmse)

In [ ]:
alphas_alt = [14.5, 14.6, 14.7, 14.8, 14.9, 15, 15.1, 15.2, 15.3, 15.4, 15.5]
alphas2 = [5e-05, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008]
e_alphas = [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007]
e_l1ratio = [0.8, 0.85, 0.9, 0.95, 0.99, 1]

In [ ]:
rIDge = make_pipeline(RobustScaler(), RidgeCV(alphas=alphas_alt, cv=kfolds))
lasso = make_pipeline(RobustScaler(), LassoCV(max_iter=1e7, alphas=alphas2, random_state=42, cv=kfolds))
elasticnet = make_pipeline(RobustScaler(), ElasticNetCV(max_iter=1e7, alphas=e_alphas, cv=kfolds, l1_ratio=e_l1ratio))                                
svr = make_pipeline(RobustScaler(), SVR(C= 20, epsilon= 0.008, gamma=0.0003,))



from sklearn.kernel_ridge import KernelRidge

kernel_reg = make_pipeline(RobustScaler(), KernelRidge(alpha=1.0, kernel='linear', degree=3, gamma=None, coef0=1, kernel_params=None))


In [ ]:
n_estimators = 9500

In [ ]:
rf = RandomForestRegressor(n_estimators=n_estimators, random_state=42)

In [ ]:
gbr = GradientBoostingRegressor(n_estimators=n_estimators, learning_rate=0.05, max_depth=4, max_features='sqrt', min_samples_leaf=15, min_samples_split=10, loss='huber', random_state =42)                             

In [ ]:
lightgbm = LGBMRegressor(objective='regression', 
                                       num_leaves=4,
                                       learning_rate=0.01, 
                                       n_estimators=n_estimators,
                                       max_bin=200, 
                                       bagging_fraction=0.75,
                                       bagging_freq=5, 
                                       bagging_seed=7,
                                       feature_fraction=0.2,
                                       feature_fraction_seed=7,
                                       verbose=-1,
                                       )

In [ ]:
xgboost = XGBRegressor(learning_rate=0.01,n_estimators=n_estimators,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:linear', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006)

In [ ]:
from catboost import CatBoostRegressor

In [ ]:
cboost = CatBoostRegressor(
    iterations=10000,  # maximum number of trees to build
    learning_rate=0.1,  # learning rate for gradient boosting
    depth=6,  # maximum depth of each tree
    l2_leaf_reg=1,  # L2 regularization coefficient
    random_seed=42,  # random seed for reproducibility
    verbose = False
)


In [ ]:
stack_gen = StackingCVRegressor(regressors= (rIDge, lasso, elasticnet,kernel_reg,cboost, gbr, xgboost, lightgbm,rf),
                                meta_regressor=xgboost,
                                use_features_in_secondary=True)

In [ ]:
print('START Fit')

print('stack_gen')
stack_gen_model = stack_gen.fit(np.array(X), np.array(y))

print('elasticnet')
elastic_model_full_data = elasticnet.fit(X, y)



print('kernel_reg')
kernel_reg_full_data = kernel_reg.fit(X, y)

print('Lasso')
lasso_model_full_data = lasso.fit(X, y)

print('RIDge')
rIDge_model_full_data = rIDge.fit(X, y)

print('Svr')
svr_model_full_data = svr.fit(X, y)

print('GradientBoosting')
gbr_model_full_data = gbr.fit(X, y)

print('xgboost')
xgb_model_full_data = xgboost.fit(X, y)

print('lightgbm')
lgb_model_full_data = lightgbm.fit(X, y)

print('catboost')
cboost_model_full_data = cboost.fit(X, y)

print('catboost')
rf_model_full_data = rf.fit(X, y)

# Blending Models

In [ ]:
def blend_models_predict(X):
    #final_predictions = model.predict(X)
    return ((0.03 * elastic_model_full_data.predict(X)) + \
            (0.03 * lasso_model_full_data.predict(X)) + \
            (0.04 * kernel_reg_full_data.predict(X)) + \
            (0.04 * rIDge_model_full_data.predict(X)) + \
            (0.04 * svr_model_full_data.predict(X)) + \
            (0.13 * gbr_model_full_data.predict(X)) + \
            (0.13 * xgb_model_full_data.predict(X)) + \
            (0.13 * lgb_model_full_data.predict(X)) + \
            (0.13 * cboost_model_full_data.predict(X))+ \
            (0.10 * cboost_model_full_data.predict(X))+ \
            #(0.4 * final_predictions.reshape((final_predictions.shape[0],)))+\
            (0.20 * stack_gen_model.predict(np.array(X))))

In [ ]:
print('RMSLE score on train data:')
print(rmsle(np.expm1(y), np.expm1(blend_models_predict(X))))

In [ ]:
blend_models_predict(X)[:2]

In [ ]:
print('Predict submission')
submission = pd.read_csv("/kaggle/input/ml-for-oceanography/sample2.csv")
submission.iloc[:,1] = np.expm1(blend_models_predict(X_sub)) #np.expm1(blend_models_predict(X_sub))

# Submission

In [ ]:
submission.to_csv("submission101(best+log1p transform).csv", index=False)

In [ ]:
submission.head()

Based on: **https://www.kaggle.com/itslek/blend-stack-lr-gb-0-10649-house-prices-v57/data?scriptVersionID=11189608**